NilsPod Tutorial 
============
## Ferienakademie 2019 
### Luca & Janis

In [18]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

NilsPod Python Lib
=======
Modules:
- session_base
- calibration_utils
- consts
- dataset
- datastream
- exceptions
- header
- legacy
- session
- utils
<br>The session and dataset module are most likely the most important ones.


Sessions
=========

A Session groups multiple Datasets from sensors. The Session module itself does not make any assumptions about how and when the datasets were recorded. It just poses an interface to manipulate the given datasets simultaneously, for instance, we can extract a certain period of interest from all sessions or calibrate certain sensors of the NilsPods. 

In [19]:
from NilsPodLib import Dataset, Session, SyncedSession, calibration_utils

FILEPATH = Path('./Data/')
#In many cases we want to include all log-files in a certain dir:
datasets = [Dataset.from_bin_file(d) for d in FILEPATH.glob('*.bin')]
session = Session(datasets)
#session = Session.from_folder_path(FILEPATH, filter_pattern='*.bin')
print('This session has {} datasets'.format(len(session.datasets)))

This session has 2 datasets


In [20]:
for ds in session.datasets:
    for name, d in ds.datastreams:
        print(f'{name} of {ds.info.sensor_id} has the length {len(d.data)}')

gyro of 9433 has the length 24100
acc of 9433 has the length 24100
gyro of e0ef has the length 24096
acc of e0ef has the length 24096
ecg of e0ef has the length 24096


### Session Operations

#### Downsampling

In [21]:
downsampled_session = session.downsample(factor=2)
for ds in downsampled_session.datasets:
    for name, d in ds.datastreams:
        print(f'{name} of {ds.info.sensor_id} has the length {len(d.data)}')

gyro of 9433 has the length 12050
acc of 9433 has the length 12050
gyro of e0ef has the length 12048
acc of e0ef has the length 12048
ecg of e0ef has the length 12048


#### Calibration
Sadly, we used two sensors that do not have any calibrations...

In [22]:
ds = session.datasets[0]
print(ds.info.sensor_id)
#your_cal = calibration_utils.find_calibrations_for_sensor(ds.info.sensor_id, folder='/Data') # Alternative: Select a callibration from a list of given calibrations
#calibrated_acc = session.calibrate_gyro(your_cal[0])
#calibrated_acc.data_as_df()[1].plot()

9433


In [23]:
calibrated_acc = session.factory_calibrate_gyro() # Set facotry calibtration settings
ax = calibrated_acc.data_as_df()[0].plot()
ax2 = calibrated_acc.data_as_df()[1].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Extraction

In [24]:
extracted = session.cut(7000, 10000,step=1) 
extracted.data_as_df()[0].plot()
extracted.data_as_df()[1].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### General Information

In [14]:
# Further you can use the Proxy Attribute `info` to access the header infos of all sensors at the same time
print(f'The included sensors are: {session.info.sensor_id}')
print(f'The samplingrates are: {session.info.sampling_rate_hz}')
print(f'The enabled sensor are: {session.info.enabled_sensors} \n')

The included sensors are: ('9433', 'e0ef')
The samplingrates are: (256.0, 256.0)
The enabled sensor are: (('gyro', 'acc'), ('gyro', 'acc', 'ecg')) 



Synchronised Session
====================
The library differentiates between synchronised and not synchronised session. Synchronized means that all datasets have the same length and the exact same counter.<br>
If your session was recorded synchronously you should use a SyncedSession. However, datasets from different measurements can be forced into a synchronized session (VALIDATE_ON_INIT=False).<br>
A SyncedSession always consits of one master and at least one slave. Hereby, the master conduces as a clock.

In [15]:
session = SyncedSession.from_folder_path(FILEPATH)

# This will also validate that all datasets are compatible to be syncronised.
# If you need to switch off this validation, you can disable it using:
SyncedSession.VALIDATE_ON_INIT = False
session = SyncedSession.from_folder_path(FILEPATH)

# For synced sessions you can get the datasets of the master and the slaves separately

print('The master of the session is', session.master.info.sensor_id)
print('The slaves of the session are', [d.info.sensor_id for d in session.slaves])

# To make use of the sync information, all datasets need to be aligned. This can be done using the `cut_to_syncregion`
# method.

cut_session = session.cut_to_syncregion()

# After this all session are aligned and the dataset counter are identical

for d in cut_session.slaves:
    if np.array_equal(d.counter, cut_session.master.counter) is True:
        print('{} has the same counter than master ({})'.format(d.info.sensor_id, cut_session.master.info.sensor_id))

The master of the session is e0ef
The slaves of the session are ['9433']
9433 has the same counter than master (e0ef)


<br>Now, all dataset have the same length and same index.<br>

In [16]:
for ds in cut_session.datasets:
    for name, d in ds.datastreams:
        print(f'{name} of {ds.info.sensor_id} has the length {len(d.data)}')

gyro of 9433 has the length 22649
acc of 9433 has the length 22649
gyro of e0ef has the length 22649
acc of e0ef has the length 22649
ecg of e0ef has the length 22649


<br>We can use all functions of normal Sessions with SyncedSessions as well. Additionally, several validations can be checked.<br>

In [17]:
print(f'All provided sessions have overlapping recording times: {cut_session._validate_overlapping_record_time()}')
print(f'All sensors had the same sampling rate: {cut_session._validate_sampling_rate()}')
print(f'There is only 1 master and all other sensors were configured as slaves: {cut_session._validate_sync_role()}')

All provided sessions have overlapping recording times: True
All sensors had the same sampling rate: True
There is only 1 master and all other sensors were configured as slaves: (True, True)
